In [1]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

load_dotenv()

model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

### Chat Model

In [10]:
# 일반 챗
messages = [
  SystemMessage('당신은 도우미 어시스턴트입니다.'),
  HumanMessage('안녕하세요! 저는 손이라고 합니다.'),
  AIMessage(content='안녕하세요 손님! 어떤 도움이 필요하신가요?'),
  HumanMessage(content='제 이름을 아시나요?')
]

ai_message = model.invoke(messages)
print(f'response : {ai_message}')
print(f'content : {ai_message.content}')

response : content='네, 당신의 이름은 손이라고 하셨습니다. 어떻게 도와드릴까요?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 60, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_51db84afab', 'id': 'chatcmpl-CQR9aBmO7iD4Qyrxumk0uEkgv9I8A', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--ba64c480-1fce-4360-8371-6ef1d30ff5e8-0' usage_metadata={'input_tokens': 60, 'output_tokens': 19, 'total_tokens': 79, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
content : 네, 당신의 이름은 손이라고 하셨습니다. 어떻게 도와드릴까요?


In [ ]:
# 스트리밍
messages = [
  SystemMessage('당신은 도우미 어시스턴트입니다.'),
  HumanMessage('안녕하세요!'),
]

for chunk in model.stream(messages):
  print(chunk.content, end="", flush=True)

안녕하세요! 어떻게 도와드릴까요?

### Prompt template

```
PromptTemplate
```
문자열 일부를 대체하는 것뿐, 내부에서 LLM을 호출하지 않음

In [ ]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""다음 요리의 레시피를 생각해 주세요.

요리명: {dish}""")

prompt_value = prompt.invoke({'dish':'카레'})
print(prompt_value.text)

다음 요리의 레시피를 생각해 주세요.

요리명: 카레


```
ChatPromptTemplate
```
채팅 형식 모뎅에 대응 시킴

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate(
  [
    ('system', '사용자가 입력한 요리의 레시피를 생각해 주세요.'),
    ('human', '{dish}'),
  ]
)

prompt_value = prompt.invoke({'dish':'카레'})
print(prompt_value)

messages=[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='카레', additional_kwargs={}, response_metadata={})]


```
MessagesPlaceholder
```
대화 이력처럼 여러 메세지가 들어가는 플레이스홀더를 두고 싶은 경우가 많을때 사용

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

prompt = ChatPromptTemplate(
  [
    ('system', '당신은 도우미 어시스턴트입니다.'),
    MessagesPlaceholder('chat_history', optional=True),
    ('human', '{input}'),
  ]
)

prompt_value = prompt.invoke(
  {
    'chat_history': [
      HumanMessage(content='안녕하세요! 저는 손이라고 합니다.'),
      AIMessage('안녕하세요, 손님! 무엇을 도와드릴까요?'),
    ],
    'input': '제 이름을 아시나요?'
  }
)
print(prompt_value)

messages=[SystemMessage(content='당신은 도우미 어시스턴트입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요! 저는 손이라고 합니다.', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 손님! 무엇을 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='제 이름을 아시나요?', additional_kwargs={}, response_metadata={})]


### Output parser

In [ ]:
from pydantic import BaseModel, Field

# 재료 목록과 순서 정의
class Recipe(BaseModel):
  ingredients: list[str] = Field(description="ingredients of the dish")
  steps: list[str] = Field(description="steps to make the dish")

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser

# 클래스를 제공하여 PydanticOutputParser를 생성
output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
# 프롬프트에 출력할 출력 형식 설명문 작성
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [36]:
from langchain_core.prompts import ChatPromptTemplate

# format_instructions를 사용한 ChatPromptTemplate 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자가 입력한 요리의 레시피를 생각해 주세요.\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

In [37]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "카레"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

=== role: system ===
사용자가 입력한 요리의 레시피를 생각해 주세요.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
카레


In [38]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)

{
  "ingredients": [
    "닭고기 500g",
    "양파 1개",
    "감자 2개",
    "당근 1개",
    "카레 가루 3큰술",
    "식용유 2큰술",
    "소금 약간",
    "후추 약간",
    "물 4컵"
  ],
  "steps": [
    "양파를 잘게 썰고, 감자와 당근은 큐브 모양으로 자릅니다.",
    "냄비에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.",
    "닭고기를 넣고 겉면이 노릇해질 때까지 볶습니다.",
    "감자와 당근을 추가하고 함께 볶습니다.",
    "카레 가루를 넣고 잘 섞은 후 물을 부어 끓입니다.",
    "끓기 시작하면 중약불로 줄이고 20분 정도 끓입니다.",
    "소금과 후추로 간을 맞춘 후 불을 끄고 5분 정도 둡니다.",
    "밥과 함께 서빙합니다."
  ]
}


In [39]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 4컵'] steps=['양파를 잘게 썰고, 감자와 당근은 큐브 모양으로 자릅니다.', '냄비에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.', '닭고기를 넣고 겉면이 노릇해질 때까지 볶습니다.', '감자와 당근을 추가하고 함께 볶습니다.', '카레 가루를 넣고 잘 섞은 후 물을 부어 끓입니다.', '끓기 시작하면 중약불로 줄이고 20분 정도 끓입니다.', '소금과 후추로 간을 맞춘 후 불을 끄고 5분 정도 둡니다.', '밥과 함께 서빙합니다.']


### StrOutputParser

In [40]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="안녕하세요. 저는 AI 어시스턴트입니다.")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

<class 'str'>
안녕하세요. 저는 AI 어시스턴트입니다.


### Chain-LangChain Expression Language (LCEL)

단순 LLM에 입출력 뿐만 아니라 처리를 연쇄적으로 하고 싶은 경우가 있음
```
- prompt template을 채우고, 그 결과를 Chat model에 제공한 후, 그 결과를 Python 객체로 변환하고 싶다.
- Zero-shot CoT 프롬프팅으로 단계별로 생각하게하고, 그 결과를 요약하고 싶다.
- LLM의 출력을 얻은 후에, 그 내요이 서비스 정책에 위반되지 않는지 확인하고 싶다.
- LLM의 출력 결과를 바탕으로 SQL을 실행하여 데이터를 분석하고 싶다.
```
이러한 처리의 연쇄를 구현한 것이 LangChain의 'Chain'

```
LCEL은 LangChain에 Chain을 기술하는 방법
```

In [41]:
# prompt와 model 연결
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate(
  [
    ('system','사용자가 입력한 요리의 레시피를 생각해 주세요.'),
    ('human', '{dish}'),
  ]
)
model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [42]:
chain = prompt | model

In [43]:
ai_message = chain.invoke({'dish':'카레'})
print(ai_message.content)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 선택 재료가 있다면 함께 손질합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다. 선택 재료가 있다면 이때 추가합니다.

5. **물 붓기**: 모든 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불에서 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞은 후, 다시 10분 정도 끓입니다. 필요에 따라 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 하루 정도 숙성시키면 더욱 맛있습니다.
- 다양한 재료를 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!


```
StrOutputParser를 체인에 추가
```

In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({'dish':'z카레'})
print(output)

Z카레는 일본식 카레의 한 종류로, 일반적으로 부드러운 카레 소스와 다양한 재료가 어우러진 요리입니다. 아래는 Z카레의 기본 레시피입니다.

### 재료
- 카레 루(일본식 카레 가루) 1팩
- 닭고기 또는 소고기 300g (한입 크기로 자른 것)
- 감자 1개 (깍둑썰기)
- 당근 1개 (깍둑썰기)
- 양파 1개 (채썰기)
- 물 4컵
- 식용유 1큰술
- 소금, 후추 (기호에 따라)

### 조리 방법
1. **재료 준비**: 모든 재료를 손질하여 준비합니다. 고기는 한입 크기로 자르고, 감자와 당근은 깍둑썰기, 양파는 채썰어 둡니다.

2. **고기 볶기**: 큰 냄비에 식용유를 두르고 중불에서 양파를 볶아 투명해질 때까지 볶습니다. 그 후, 고기를 넣고 겉면이 노릇해질 때까지 볶습니다.

3. **채소 추가**: 볶은 고기에 감자와 당근을 추가하고, 1-2분 더 볶습니다.

4. **물 붓기**: 냄비에 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고, 뚜껑을 덮고 약 15-20분간 끓입니다. 이때 재료가 부드러워질 때까지 조리합니다.

5. **카레 루 추가**: 카레 루를 조금씩 넣으면서 잘 저어줍니다. 카레 루가 완전히 녹을 때까지 저어주고, 다시 끓입니다.

6. **간 맞추기**: 소금과 후추로 간을 맞추고, 원하는 농도가 될 때까지 약한 불에서 5-10분 더 끓입니다.

7. **서빙**: 완성된 Z카레를 밥 위에 얹어 서빙합니다. 기호에 따라 피클이나 샐러드를 곁들여도 좋습니다.

맛있게 드세요!


```
PydanticOutputParser를 사용한 체인
```

In [6]:
# Recipe 클래스 정의
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel,Field

class Recipe(BaseModel):
  ingredients: list[str] = Field(description="ingredients of the dish")
  steps: list[str] = Field(description="steps to make the dish")

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [7]:
# 체인으로 연결
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate(
  [
    ('system', '사용자가 입력한 요리의 레시피를 생각해 주세요.\n\n{format_instructions}'),
    ('human', '{dish}'),
  ]
)

prompt_with_format_instructions = prompt.partial(
  format_instructions=output_parser.get_format_instructions()
)

model = ChatOpenAI(model='gpt-4o-mini', temperature=0).bind(
  response_format={'type': 'json_object'}
)

In [8]:
chain = prompt_with_format_instructions | model | output_parser

recipe = chain.invoke({'dish':'카레'})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 4컵'] steps=['양파를 잘게 썰고, 감자와 당근은 큐브 모양으로 자릅니다.', '냄비에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶습니다.', '닭고기를 넣고 겉면이 노릇해질 때까지 볶습니다.', '감자와 당근을 추가하고 함께 볶습니다.', '카레 가루를 넣고 잘 섞은 후 물을 부어 끓입니다.', '끓기 시작하면 중약불로 줄이고 20분 정도 끓입니다.', '소금과 후추로 간을 맞춘 후 불을 끄고 5분 정도 둡니다.', '밥과 함께 따뜻하게 서빙합니다.']


### RAG (Retrieval-Augumented Generation)

```
Document loader
```

In [11]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".md")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

41


```
Document transformer
```

In [12]:
from langchain_text_splitters import CharacterTextSplitter

text_spliter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_spliter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 1329, which is longer than the specified 1000
Created a chunk of size 1422, which is longer than the specified 1000
Created a chunk of size 1166, which is longer than the specified 1000


110


```
Embedding model
```

In [13]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model='text-embedding-3-small')

In [15]:
query = 'AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?'

vector = embeddings.embed_query(query)
print(len(vector))
print(vector)

1536
[0.0063845072872936726, 0.008431448601186275, 0.023545239120721817, -0.027140924707055092, 0.03550197556614876, 0.006292448844760656, -0.0015609280671924353, 0.0173394326120615, 0.011491029523313046, -0.025689654052257538, -0.0010004560463130474, -0.002333946293219924, -0.009449503384530544, -0.008713037706911564, -0.015205848962068558, 0.04873669520020485, 0.0074783749878406525, -0.015043393708765507, -0.029198696836829185, 0.0004998896038159728, 0.025364741683006287, 0.033032648265361786, -0.026447780430316925, 0.05055619776248932, 0.025776296854019165, -0.04739372804760933, -0.017512720078229904, 0.034722186625003815, -0.013104756362736225, -0.10700412094593048, -0.01300728227943182, -0.05449845641851425, -0.025386402383446693, 0.03275106102228165, -0.015292491763830185, 0.05250566825270653, 0.04566086828708649, 0.009666111320257187, -0.011491029523313046, -0.01996038481593132, 0.012292477302253246, -0.004088467452675104, 0.015292491763830185, 0.004256337881088257, 0.0301517695

```
Vector store
```

In [16]:
from langchain_chroma import Chroma

db = Chroma.from_documents(docs, embeddings)

In [17]:
retriever = db.as_retriever()

In [18]:
query = 'AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?'

context_docs = retriever.invoke(query)
print(f'len = {len(context_docs)}')
first_doc = context_docs[0]
print(f'metadata = {first_doc}')
print(first_doc.page_content)

len = 4
metadata = page_content='* [OpenAI](https://pypi.org/project/langchain-openai/)
* [Anthropic](https://pypi.org/project/langchain-anthropic/)
* [Ollama](https://pypi.org/project/langchain-ollama/)
* [DeepSeek](https://pypi.org/project/langchain-deepseek/)
* [xAI](https://pypi.org/project/langchain-xai/)
* and more

Most integrations have been moved to their own repositories for improved versioning, dependency management, collaboration, and testing. This includes packages from popular providers such as [Google](https://github.com/langchain-ai/langchain-google) and [AWS](https://github.com/langchain-ai/langchain-aws). Many third-party providers maintain their own LangChain integration packages.

For a full list of all LangChain integrations, please refer to the [LangChain Integrations documentation](https://docs.langchain.com/oss/python/integrations/providers).' metadata={'file_name': 'README.md', 'source': 'libs\\README.md', 'file_type': '.md', 'file_path': 'libs\\README.md'}
* [

### LCEL를 사용한 RAG Chain 구현

In [23]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 바탕으로 질문에 답변해 주세요.

문맥: """
{context}
"""

  질문: {question}
''')

model = ChatOpenAI(model='gpt-4o-mini', temperature=0)

In [26]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
  {'context' : retriever, 'question' : RunnablePassthrough()}
  | prompt
  | model
  | StrOutputParser()
)

output = chain.invoke(query)
print(output)

문맥에는 AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader에 대한 구체적인 정보는 포함되어 있지 않습니다. 그러나 AWS와 관련된 LangChain 통합 패키지가 언급되어 있으므로, AWS S3와 관련된 Document loader가 있을 가능성이 있습니다. 더 자세한 정보는 LangChain Integrations 문서에서 확인할 수 있습니다.


### LangChain Expresssion Language(LCEL) 심층

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [28]:
prompt_value = prompt.invoke({"dish": "카레"})
ai_message = model.invoke(prompt_value)
output = output_parser.invoke(ai_message)

print(output)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 선택 재료가 있다면 함께 손질합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 볶아줍니다. 양파가 투명해질 때까지 볶습니다.

3. **고기 추가**: 손질한 고기를 넣고 겉면이 익을 때까지 볶습니다. 이때 마늘과 생강을 추가하면 향이 더 좋습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다. 다른 채소도 이 단계에서 추가할 수 있습니다.

5. **물 붓기**: 모든 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불에서 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞어줍니다. 원하는 농도에 따라 물을 추가할 수 있습니다. 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 5분 정도 뜸을 들인 후 그릇에 담아냅니다.

### 서빙
밥과 함께 서빙하면 좋습니다. 또한, 피클이나 요거트와 함께 곁들여 먹으면 더욱 맛있습니다.

맛있게 드세요!


```
invoke
```

In [29]:
chain = prompt | model | output_parser

output = chain.invoke({"dish": "카레"})
print(output)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 돼지고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 코코넛 밀크 또는 물 400ml
- 식용유 2 큰술
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 선택 재료도 적당한 크기로 썰어줍니다.

2. **양파 볶기**: 큰 팬이나 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 볶아줍니다. 양파가 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다. 이때 마늘과 생강을 추가하면 향이 더 좋습니다.

4. **채소 추가**: 고기가 익으면 감자와 당근을 넣고 함께 볶아줍니다. 다른 채소도 이 단계에서 추가할 수 있습니다.

5. **카레 가루 추가**: 카레 가루를 넣고 잘 섞어줍니다. 향신료가 잘 섞이도록 1-2분 정도 볶아줍니다.

6. **액체 추가**: 코코넛 밀크 또는 물을 부어줍니다. 끓어오르면 불을 줄이고 뚜껑을 덮고 20-30분 정도 끓입니다. 중간에 간을 보고 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지고 재료가 부드러워지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 하루 정도 숙성시키면 더욱 맛있습니다.
- 다양한 재료를 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!


```
stream
```

In [30]:
chain = prompt | model | output_parser

for chunk in chain.stream({"dish": "카레"}):
    print(chunk, end="", flush=True)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 선택 재료가 있다면 함께 손질합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 볶아줍니다. 양파가 투명해질 때까지 볶습니다.

3. **고기 추가**: 손질한 고기를 넣고 겉면이 익을 때까지 볶습니다. 이때 마늘과 생강을 추가하면 향이 더 좋습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다. 다른 채소도 이 단계에서 추가할 수 있습니다.

5. **물 붓기**: 모든 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불에서 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞어줍니다. 원하는 농도에 따라 물을 추가할 수 있습니다. 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 시간이 지날수록 맛이 깊어지므로, 하루 정도 숙성시키면 더욱 맛있습니다.
- 다양한 재료를 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!

```
batch
```


In [31]:
chain = prompt | model | output_parser

outputs = chain.batch([{"dish": "카레"}, {"dish": "우동"}])
print(outputs)

['카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.\n\n### 재료\n- 고기 (닭고기, 소고기, 돼지고기 등) 300g\n- 양파 1개\n- 감자 1개\n- 당근 1개\n- 카레 가루 2-3 큰술\n- 코코넛 밀크 또는 물 400ml\n- 식용유 2 큰술\n- 소금, 후추 약간\n- 선택 재료: 마늘, 생강, 피망, 버섯 등\n\n### 조리 방법\n1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 선택 재료도 적당한 크기로 썰어줍니다.\n\n2. **양파 볶기**: 큰 팬이나 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 볶아줍니다. 양파가 투명해질 때까지 볶습니다.\n\n3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.\n\n4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다. 이때 마늘과 생강을 추가하면 향이 더해집니다.\n\n5. **카레 가루 추가**: 카레 가루를 넣고 잘 섞어줍니다. 향신료가 잘 섞이도록 1-2분 정도 볶아줍니다.\n\n6. **액체 추가**: 코코넛 밀크 또는 물을 부어줍니다. 끓어오르면 불을 줄이고 뚜껑을 덮고 20-30분 정도 끓입니다. 중간에 간을 보고 소금과 후추로 간을 맞춥니다.\n\n7. **완성**: 모든 재료가 부드럽게 익으면 불을 끄고, 원하는 경우 신선한 고수를 올려서 장식합니다.\n\n8. **서빙**: 밥이나 난과 함께 따뜻하게 서빙합니다.\n\n맛있게 드세요! 카레는 다양한 변형이 가능하니, 취향에 맞게 재료를 추가하거나 조절해 보세요.', '우동은 일본의 전통적인 면 요리로, 쫄깃한 면과 다양한 재료가 어우러져 맛있는 국물 요리입니다. 아래는 기본적인 우동 레시피입니다.\n\n### 재료\n- 우동 면 2인분\n- 물 4컵\n- 다시마 10cm 조각\n- 가쓰오부시 1컵\n- 간장 4큰술\n- 미림 2큰술\n- 소금 약간\n- 대파 (썰어서) 1대\n

### chain과 chain을 연결

In [37]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

output_parser = StrOutputParser()

In [38]:
cot_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자의 질문에 단계적으로 답변하세요."),
        ("human", "{question}"),
    ]
)

cot_chain = cot_prompt | model | output_parser

In [41]:
summarize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "단계적으로 생각한 답변에서 결론인 답만 출력하세요."),
        ("human", "{text}"),
    ]
)

summarize_chain = summarize_prompt | model | output_parser

In [42]:
cot_summarize_chain = cot_chain | summarize_chain
output = cot_summarize_chain.invoke({"question": "10 + 2 * 3"})
print(output)

16


### Advanced RAG

```
단순 RAG
```

In [43]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".md")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",
    repo_path="./langchain",
    branch="master",
    file_filter=file_filter,
)

documents = loader.load()
print(len(documents))

41


In [44]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
db = Chroma.from_documents(documents, embeddings)

In [48]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''
다음 문맥만을 고려해 질문에 답하세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model='gpt-4o-mini', temperature=0)
retriever = db.as_retriever()

chain = {
  'question': RunnablePassthrough(),
  'context': retriever
} | prompt | model | StrOutputParser()

chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 LLM(대형 언어 모델) 기반 애플리케이션을 구축하기 위한 프레임워크입니다. 이 프레임워크는 상호 운용 가능한 구성 요소와 제3자 통합을 연결하여 AI 애플리케이션 개발을 간소화하며, 기술이 발전함에 따라 미래에 대비할 수 있는 결정을 내리는 데 도움을 줍니다.\n\nLangChain을 사용하면 개발자는 모델, 임베딩, 벡터 저장소 등 다양한 요소를 위한 표준 인터페이스를 통해 LLM 기반 애플리케이션을 구축할 수 있습니다. 이 프레임워크는 독립적으로 사용할 수 있지만, LangChain 제품과 원활하게 통합되어 LLM 애플리케이션을 구축할 때 전체 도구 세트를 제공합니다.\n\n주요 기능으로는 실시간 데이터 증강, 모델 상호 운용성, 에이전트 평가 및 관찰 가능성, 복잡한 작업을 처리할 수 있는 에이전트 구축 등이 있습니다. LangChain은 다양한 데이터 소스와 외부 시스템에 쉽게 연결할 수 있는 기능을 제공하며, 개발자가 애플리케이션의 요구에 맞는 최적의 모델을 찾을 수 있도록 지원합니다.'

```
HyDE(Hypothetical Document Embeddings)
```

In [58]:
hypothetical_prompt = ChatPromptTemplate.from_template("""
다음 질문에 한 문장으로 답하세요.

질문: {question}
""")

# 가상의 답변 생성성
hypothetical_chain = hypothetical_prompt | model | StrOutputParser()

In [59]:
hyde_rag_chain = {
  'question': RunnablePassthrough(),
  'context': hypothetical_chain | retriever, 
} | prompt | model | StrOutputParser()

chain.invoke("LangChain의 개요를 알려줘")

'LangChain은 LLM(대형 언어 모델) 기반 애플리케이션을 구축하기 위한 프레임워크입니다. 이 프레임워크는 상호 운용 가능한 구성 요소와 제3자 통합을 연결하여 AI 애플리케이션 개발을 간소화하며, 기술이 발전함에 따라 미래에 대비할 수 있는 결정을 내리는 데 도움을 줍니다.\n\nLangChain을 사용하면 개발자는 모델, 임베딩, 벡터 저장소 등 다양한 요소에 대한 표준 인터페이스를 통해 LLM을 활용한 애플리케이션을 쉽게 구축할 수 있습니다. 이 프레임워크는 독립적으로 사용할 수 있지만, LangChain 제품과 원활하게 통합되어 LLM 애플리케이션을 구축할 때 전체 도구 세트를 제공합니다.\n\n주요 기능으로는 실시간 데이터 증강, 모델 상호 운용성, 에이전트 평가 및 관찰 가능성, 복잡한 작업을 처리할 수 있는 에이전트 구축 등이 있습니다. LangChain은 다양한 데이터 소스와 외부 시스템에 쉽게 연결할 수 있는 통합 라이브러리를 제공하며, 개발자가 애플리케이션의 요구에 맞는 최적의 모델을 실험하고 빠르게 적응할 수 있도록 지원합니다.'

```
복수 검색 쿼리 생성
```

In [63]:
from pydoc import describe
from pydantic import BaseModel, Field

class QueryGenerationOutput(BaseModel):
  queries: list[str] = Field(..., description='검색 쿼리 목록')

query_generation_prompt = ChatPromptTemplate.from_template("""\
질문에 대해 벡터 데이터베이스에서 관련 문서를 검색하기 위한
3개의 서로 다른 검색 쿼리를 생성하세요.
거리 기반 유사성 검색의 한계를 극복하기 위해
사용자의 질문에 대해 여러 관점을 제공하는 것이 목표입니다.

질문: {question}
""")

query_generation_chain = (
  query_generation_prompt 
  | model.with_structured_output(QueryGenerationOutput)
  | (lambda x: x.queries)
)

In [66]:
multi_query_rag_chain = {
  'question': RunnablePassthrough(),
  'context': query_generation_chain | retriever.map(),
} | prompt | model | StrOutputParser()

multi_query_rag_chain.invoke("Langchain의 개요를 알려줘")

'LangChain은 LLM(대형 언어 모델) 기반 애플리케이션을 구축하기 위한 프레임워크입니다. 이 프레임워크는 상호 운용 가능한 구성 요소와 제3자 통합을 연결하여 AI 애플리케이션 개발을 간소화하고, 기술이 발전함에 따라 미래에 대비할 수 있도록 돕습니다. LangChain을 사용하면 모델, 임베딩, 벡터 저장소 등을 위한 표준 인터페이스를 통해 LLM으로 구동되는 애플리케이션을 쉽게 구축할 수 있습니다.\n\n주요 기능으로는 실시간 데이터 증강, 모델 상호 운용성, 그리고 다양한 데이터 소스와 시스템에 쉽게 연결할 수 있는 기능이 있습니다. LangChain은 독립적으로 사용할 수 있지만, LangChain 제품과 원활하게 통합되어 LLM 애플리케이션을 구축하는 데 필요한 전체 도구 모음을 제공합니다. \n\n또한, LangChain은 LangSmith, LangGraph, LangGraph Platform과 같은 다양한 도구와 함께 사용할 수 있어, 복잡한 작업을 처리할 수 있는 에이전트를 구축하고, 성능을 개선하며, 에이전트를 배포하고 확장하는 데 도움을 줍니다.'